In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import time
from datetime import datetime

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)




# Wczytywanie danych

In [ ]:
import os
MASKS_PATH = '/qarr/studia/magister/datasets/FlickrLogos-v2/classes/masks/'
INPUT_PATH = '/qarr/studia/magister/datasets/FlickrLogos-v2/classes/jpg/'

classes = [o for o in os.listdir(INPUT_PATH) if os.path.isdir(INPUT_PATH + '/' + o)]
classes = [o for o in classes if o != 'no-logo']

In [ ]:
images = dict()
targets = dict()
queries = dict()
start_time = time.time()

def rescale(nparray, scale=255.0):
    return np.array(nparray, dtype=np.float32)/scale

for c in classes:
    root_input = INPUT_PATH + '/' + c 
    root_masks = MASKS_PATH + '/' + c
    images[c] = list()
    targets[c] = list()
    queries[c] = list()
    
    for f in os.listdir(root_input):
        img = cv2.imread(f'{root_input}/{f}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(f'{root_masks}/{f}.mask.merged.png', cv2.IMREAD_GRAYSCALE)
        bboxes = []
        
        with open(f'{root_masks}/{f}.bboxes.txt') as csvfile:
            bboxread = csv.reader(csvfile, delimiter=' ')
            next(bboxread)
            for row in bboxread:
                bboxes.append(row)
                
        for bbox in bboxes:
            x,y,w,h = [int(i) for i in bbox]
            imgslice = img[y:y+h, x:x+w]
            imgslice = cv2.resize(imgslice, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
            queries[c].append(rescale(imgslice, 255.0))
            # Biore tylko pierwszy z dostepnych bbox na obrazku
            break 
            
        img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
        mask = cv2.resize(mask, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    
        images[c].append(rescale(img, 255.0))
        targets[c].append(rescale(mask, 255.0))


end_time = time.time()

print(f'Time taken: {end_time-start_time} seconds')


# Generator danych

In [ ]:
batch_size = 24

In [ ]:
rnd = np.random.RandomState(13371337)

nclasses = len(classes)
nlogos = sum([len(images[c]) for c in classes])//nclasses
all_cases = nclasses*nlogos*(nlogos-1)
valid_cases = ((all_cases//batch_size)//10)*batch_size
valid_unique_n = 2
valid_unique_pairs = nclasses*2*np.sum(range(nlogos-1, nlogos-valid_unique_n-1, -1))

train_data_permutations = np.zeros((all_cases-valid_cases, 3), dtype=np.int8)
valid_data_permutations = np.zeros((valid_cases, 3), dtype=np.int8)

skips = np.sort(rnd.choice(all_cases-valid_cases, size=valid_cases-valid_unique_pairs+1, replace=False))
skips[-1] = all_cases

trainIt = 0
validIt = 0
skipIt = 0
for c_i in range(nclasses):
    valid_unique = rnd.choice(nlogos, size=valid_unique_n, replace=False)
    for n_i in range(nlogos):
        for l_i in range(nlogos):
            if n_i == l_i:
                continue
            if n_i in valid_unique or l_i in valid_unique:
                valid_data_permutations[validIt] = (c_i, n_i, l_i)
                validIt += 1
            elif skips[skipIt] == trainIt:
                valid_data_permutations[validIt] = (c_i, n_i, l_i)
                validIt += 1
                skipIt += 1
            else:
                train_data_permutations[trainIt] = (c_i, n_i, l_i)                
                trainIt += 1

In [ ]:
train_data_permutations = rnd.permutation(train_data_permutations)
train_data_permutations = train_data_permutations[:(len(train_data_permutations)//5//batch_size)*batch_size]
valid_data_permutations = rnd.permutation(valid_data_permutations)
valid_data_permutations = valid_data_permutations[:(len(valid_data_permutations)//4//batch_size)*batch_size]

In [ ]:
def dataset_permutations_generator(batch_size, data_permutations, repeat=True, shuffle=True):
    s = 0
    outimage = []
    outquery = []
    outtarget = []
    loop = True
    while loop:
        if shuffle:
            data_permutations = np.random.permutation(data_permutations)
        for class_number, image_number, query_number in data_permutations:
            c = classes[class_number]
            outimage.append(images[c][image_number])
            outquery.append(queries[c][query_number])
            outtarget.append(targets[c][image_number])
            s += 1
            if s >= batch_size:
                s = 0
                yield (np.reshape(outimage, (batch_size, 256, 256, 3)),
                       np.reshape(outquery, (batch_size, 64, 64, 3))
                      ), np.reshape(outtarget, (batch_size, 256, 256, 1))
                outimage = []
                outquery = []
                outtarget = []
    loop = repeat

In [ ]:
def dataset_permutations_generator_dummy(batch_size, data_permutations, repeat=True, shuffle=True):
    s = 0
    outimage = []
    outquery = []
    outtarget = []
    loop = True
    while loop:
        if shuffle:
            data_permutations = np.random.permutation(data_permutations)
        for class_number, image_number, query_number in data_permutations:
            c = classes[class_number]
            outimage.append(images[c][image_number])
            outquery.append(queries[c][query_number])
            outtarget.append(targets[c][image_number])
            s += 1
            if s >= batch_size:
                s = 0
                yield (np.reshape(outimage, (batch_size, 256, 256, 3))), np.reshape(outtarget, (batch_size, 256, 256, 1))
                outimage = []
                outquery = []
                outtarget = []
    loop = repeat

## Tensorflow datasets

In [ ]:
unetValidDataset = tf.data.Dataset.from_generator(dataset_permutations_generator_dummy,
                                             args=[batch_size, valid_data_permutations],
                                             output_types=((tf.float32), tf.float32),
                                             output_shapes=(((batch_size, 256,256,3)),# (batch_size, 64,64,3)),
                                                          (batch_size, 256,256,1))
                                            )

In [ ]:
unetTrainDataset = tf.data.Dataset.from_generator(dataset_permutations_generator_dummy,
                                             args=[batch_size, train_data_permutations],
                                             output_types=((tf.float32), tf.float32),
                                             output_shapes=(((batch_size, 256,256,3)),# (batch_size, 64,64,3)),
                                                          (batch_size, 256,256,1))
                                            )

# Model

 ## Stałe i parametry warstw splotowych

In [ ]:
weightDecay = 0.0005
momentum = 0.9
learningRate = 0.04
L2penalty = weightDecay
experimentName = ""

In [ ]:
convOptions = {
    "strides": 1,
    "padding": 'SAME', 
    "activation": tf.nn.relu,
    #"kernel_regularizer": regularizers.l2(L2penalty),
    #"bias_regularizer": regularizers.l2(L2penalty),
    "kernel_initializer": tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None),
    #"bias_initalizer": tf.keras.initializers.Zeros()
}


convTransOptions = {
    "strides": (2,2),
    "padding": 'SAME', 
    "activation": tf.nn.relu,
    #"kernel_regularizer": regularizers.l2(L2penalty),
    #"bias_regularizer": regularizers.l2(L2penalty),
    "kernel_initializer": tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None),
    #"bias_initalizer": tf.keras.initializers.Zeros()
}

maxPoolOptions = {
    "pool_size": 2,
    "strides": 2,
    "padding": 'SAME'
}

In [ ]:
tf.keras.backend.clear_session()

### Część transenkodera

In [ ]:
LInputTarget = tf.keras.Input(dtype = tf.float32, shape = [256, 256, 3], name = 'Target')
layersEncoder = [LInputTarget]
transcoderInputs = []
with tf.name_scope("Encoder"):
    filtersNumber=[64, 64, None, 128, 128, None, 256, 256, None, 512, 512, None, 512, 512, None]
    for fn in filtersNumber:
        if fn is None:
            layersEncoder.append(tf.keras.layers.MaxPool2D(**maxPoolOptions)(layersEncoder[-1]))
            transcoderInputs.append(layersEncoder[-1])
        else:
            layersEncoder.append(tf.keras.layers.Conv2D(filters = fn, kernel_size=3, **convOptions)(layersEncoder[-1]))
    encoderOutput = layersEncoder[-1]
    
    modelEncoder = tf.keras.Model(
        inputs=LInputTarget, 
        outputs=encoderOutput,
        name="Encoder model"
    )

### Część conditional branch

In [ ]:
LInputQuery  = tf.keras.Input(dtype = tf.float32, shape = [64, 64, 3], name = 'Query')
layersConditionalEncoder = [LInputQuery]
with tf.name_scope("Conditional"):
    filtersNumber=[32, 32, None, 64, 64, None, 128, None, 256, None, 512, None]
    for fn in filtersNumber:
        if fn is None:
            layersConditionalEncoder.append(
                tf.keras.layers.MaxPool2D(**maxPoolOptions)(layersConditionalEncoder[-1])
            )
        else:
            layersConditionalEncoder.append(
                tf.keras.layers.Conv2D(filters = fn, kernel_size=3, **convOptions)(layersConditionalEncoder[-1])
            )
    layersConditionalEncoder.append(
                tf.keras.layers.Conv2D(filters = 512, 
                                       kernel_size=2, 
                                       strides=2, 
                                       **{k:v for k,v in convOptions.items() if k != 'strides'}
                                      )(layersConditionalEncoder[-1])
            )
    conditionalEncoderOutput = layersConditionalEncoder[-1]

    modelConditional = tf.keras.Model(
        inputs=LInputQuery,
        outputs=conditionalEncoderOutput, 
        name="Latent Representation Encoder"
    )

### Część transdekodera

In [ ]:
with tf.name_scope("Transcoder"):
    layersTransDecoder = []
    upsampledLayers = []
    tiles = [(8,8), (16,16), (32, 32), (64, 64), (128,128)]
    filters = [(None, 512, 512, 512), 
               (512, 512, 512, 512), 
               (256, 256, 256, 256), 
               (128, 128, 128, 128),
               (64, 64, 64, 64)]
    
    for tile, encodedInput, fs in zip(tiles, reversed(transcoderInputs), filters):
        # Tiling output from conditional encoder
        #layersTransDecoder.append(tf.keras.layers.UpSampling2D(size=tile)(conditionalEncoderOutput))
        # Concatenating tiled output with reverse order of encoder MaxPool layers
        layersTransDecoder.append(encodedInput)
        # Flattening the concatenation with 1x1 conv if needed and joining with last cycle's result
        if fs[0] is not None:
            layersTransDecoder.append(tf.keras.layers.Conv2D(fs[0], kernel_size=1, **convOptions)(layersTransDecoder[-1]))
            layersTransDecoder.append(tf.keras.layers.Concatenate()([layersTransDecoder[-1], upsampledLayers[-1]]))
        # Transdecoding encoded values with Conv2D layers
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[1], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[2], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        # Upsampling with transposed convolution filters, saving the layer for next cycle merging
        layersTransDecoder.append(tf.keras.layers.Conv2DTranspose(fs[3], kernel_size=3, **convTransOptions)(layersTransDecoder[-1]))
        upsampledLayers.append(layersTransDecoder[-1])
        
    unetOutput = tf.keras.layers.Conv2D(filters = 1, kernel_size=3, **convOptions, name="Output")(layersTransDecoder[-1])
    layersTransDecoder.append(unetOutput)

    modelUnet = tf.keras.Model(inputs=[LInputTarget], outputs=[unetOutput], name="Unet")

## Custom gradients logging callback

In [ ]:
class GradientsLoggerTBCallback(tf.keras.callbacks.TensorBoard):
    def __init__(self, gradient_reference, *args, **kwargs):
        super(GradientsLoggerTBCallback, self).__init__(*args, **kwargs)
        self._gradient_ref = gradient_reference
        self._epoch = 1
        self.once = True
        
    def _get_gradient(self):
        with tf.GradientTape() as tape:
            y_pred = self.model(self._gradient_ref[0], training=True)  # Forward pass
            loss = self.model.compiled_loss(y_true=self._gradient_ref[1], y_pred=y_pred)

        # Compute gradients
        trainable_vars = self.model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        return gradients

    def _log_gradients(self, epoch):
        writer = self._get_writer(self._train_run_name)
        gradients = self._get_gradient()

        with writer.as_default():
            # Getting names from model.trainable_weights
            for weights, grads in zip(self.model.trainable_weights, gradients):
                tf.summary.histogram(
                    weights.name.replace(':', '_') + '_grads', data=grads, step=epoch)
        writer.flush()

    def on_epoch_end(self, epoch, logs=None):
        super(GradientsLoggerTBCallback, self).on_epoch_end(epoch, logs=logs)
        
        self._epoch += 1
        if self.histogram_freq and epoch % self.histogram_freq == 0:
            self._log_gradients(epoch)
    
    #def on_train_batch_end(self, batch, logs=None):
        #if self.histogram_freq and self._epoch % self.histogram_freq == 0 and self.once:
            #print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))
            #print(self.__dir__())
            #print(logs)
            #self.once = False

# Eksperyment 1.

In [ ]:
experimentName = "exp1_wo_cond"

## Kompilacja modelu 

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learningRate, momentum=momentum, nesterov=False, name="SGD") 
# weight decay 0.0005 by L2

modelUnet.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, 
                                                          label_smoothing=0, 
                                                          reduction="auto", 
                                                          name="binary_crossentropy"),
                  metrics=[tf.keras.metrics.BinaryCrossentropy(name="binary_crossentropy_metric")]
)

## Uczenie

In [ ]:
# Create a TensorBoard callback
logs = "/qarr/studia/magister/models/logs/" + experimentName + datetime.now().strftime("%Y%m%d-%H%M%S")

try:
    tboard_callback = GradientsLoggerTBCallback(list(unetValidDataset.take(1))[0],
                                                                   log_dir = logs,
                                                                   histogram_freq = 1)
                                                                   #profile_batch = '1,3')
except AlreadyExistsError:
    print("Already exists, skipping")

In [ ]:
modelUnet.fit(unetTrainDataset, 
              epochs=7, 
              steps_per_epoch=len(train_data_permutations)//batch_size, 
              validation_data=unetValidDataset,
              validation_steps=valid_cases//batch_size,
              callbacks=[tboard_callback]#, callbackCheckpoint]
             ) 

# Eksperyment 2.

In [ ]:
experimentName = "exp2_wo_cond_2layer"
tf.keras.backend.clear_session()

In [ ]:
LInputTarget = tf.keras.Input(dtype = tf.float32, shape = [256, 256, 3], name = 'Target')
layersEncoder = [LInputTarget]
transcoderInputs = []
with tf.name_scope("Encoder"):
    filtersNumber=[64, 64, None, 128, 128, None]
    for fn in filtersNumber:
        if fn is None:
            layersEncoder.append(tf.keras.layers.MaxPool2D(**maxPoolOptions)(layersEncoder[-1]))
            transcoderInputs.append(layersEncoder[-1])
        else:
            layersEncoder.append(tf.keras.layers.Conv2D(filters = fn, kernel_size=3, **convOptions)(layersEncoder[-1]))
    encoderOutput = layersEncoder[-1]
    
    modelEncoder = tf.keras.Model(
        inputs=LInputTarget, 
        outputs=encoderOutput,
        name="Encoder model"
    )

In [ ]:
with tf.name_scope("Transcoder"):
    layersTransDecoder = []
    upsampledLayers = []
    filters = [(None, 128, 128, 128),
               (64, 64, 64, 64)]
    
    for encodedInput, fs in zip(reversed(transcoderInputs), filters):
        # Concatenating tiled output with reverse order of encoder MaxPool layers
        layersTransDecoder.append(encodedInput)
        # Flattening the concatenation with 1x1 conv if needed and joining with last cycle's result
        if fs[0] is not None:
            layersTransDecoder.append(tf.keras.layers.Conv2D(fs[0], kernel_size=1, **convOptions)(layersTransDecoder[-1]))
            layersTransDecoder.append(tf.keras.layers.Concatenate()([layersTransDecoder[-1], upsampledLayers[-1]]))
        # Transdecoding encoded values with Conv2D layers
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[1], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[2], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        # Upsampling with transposed convolution filters, saving the layer for next cycle merging
        layersTransDecoder.append(tf.keras.layers.Conv2DTranspose(fs[3], kernel_size=3, **convTransOptions)(layersTransDecoder[-1]))
        upsampledLayers.append(layersTransDecoder[-1])
        
    unetOutput = tf.keras.layers.Conv2D(filters = 1, kernel_size=3, **convOptions, name="Output")(layersTransDecoder[-1])
    layersTransDecoder.append(unetOutput)

    modelUnetEx2 = tf.keras.Model(inputs=[LInputTarget], outputs=[unetOutput], name="Unet")

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learningRate, momentum=momentum, nesterov=False, name="SGD") 
# weight decay 0.0005 by L2

modelUnetEx2.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, 
                                                          label_smoothing=0, 
                                                          reduction="auto", 
                                                          name="binary_crossentropy"),
                  metrics=[tf.keras.metrics.BinaryCrossentropy(name="binary_crossentropy_metric")]
)

In [ ]:
# Create a TensorBoard callback
logs = "/qarr/studia/magister/models/logs/" + experimentName + datetime.now().strftime("%Y%m%d-%H%M%S")

try:
    tboard_callback = GradientsLoggerTBCallback(list(unetValidDataset.take(1))[0],
                                                                   log_dir = logs,
                                                                   histogram_freq = 1)
except AlreadyExistsError:
    print("Already exists, skipping")

In [ ]:
modelUnetEx2.fit(unetTrainDataset, 
              epochs=3, 
              steps_per_epoch=len(train_data_permutations)//batch_size, 
              validation_data=unetValidDataset,
              validation_steps=valid_cases//batch_size,
              callbacks=[tboard_callback]#, callbackCheckpoint]
             ) 

# Eksperyment 3.

In [ ]:
experimentName = "exp3_wo_cond_1layer"
tf.keras.backend.clear_session()

In [ ]:
LInputTarget = tf.keras.Input(dtype = tf.float32, shape = [256, 256, 3], name = 'Target')
layersEncoder = [LInputTarget]
transcoderInputs = []
with tf.name_scope("Encoder"):
    filtersNumber=[64, 64, None]
    for fn in filtersNumber:
        if fn is None:
            layersEncoder.append(tf.keras.layers.MaxPool2D(**maxPoolOptions)(layersEncoder[-1]))
            transcoderInputs.append(layersEncoder[-1])
        else:
            layersEncoder.append(tf.keras.layers.Conv2D(filters = fn, kernel_size=3, **convOptions)(layersEncoder[-1]))
    encoderOutput = layersEncoder[-1]
    
    modelEncoder = tf.keras.Model(
        inputs=LInputTarget, 
        outputs=encoderOutput,
        name="Encoder model"
    )

In [ ]:
with tf.name_scope("Transcoder"):
    layersTransDecoder = []
    upsampledLayers = []
    filters = [(None, 64, 64, 64)]
    
    for encodedInput, fs in zip(reversed(transcoderInputs), filters):
        # Concatenating tiled output with reverse order of encoder MaxPool layers
        layersTransDecoder.append(encodedInput)
        # Flattening the concatenation with 1x1 conv if needed and joining with last cycle's result
        if fs[0] is not None:
            layersTransDecoder.append(tf.keras.layers.Conv2D(fs[0], kernel_size=1, **convOptions)(layersTransDecoder[-1]))
            layersTransDecoder.append(tf.keras.layers.Concatenate()([layersTransDecoder[-1], upsampledLayers[-1]]))
        # Transdecoding encoded values with Conv2D layers
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[1], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        layersTransDecoder.append(tf.keras.layers.Conv2D(fs[2], kernel_size=3, **convOptions)(layersTransDecoder[-1]))
        # Upsampling with transposed convolution filters, saving the layer for next cycle merging
        #layersTransDecoder.append(tf.keras.layers.Conv2DTranspose(fs[3], kernel_size=3, **convTransOptions)(layersTransDecoder[-1]))
        layersTransDecoder.append(tf.keras.layers.UpSampling2D(size=(2,2))(layersTransDecoder[-1]))
        upsampledLayers.append(layersTransDecoder[-1])
        
    unetOutput = tf.keras.layers.Conv2D(filters = 1, kernel_size=3, **convOptions, name="Output")(layersTransDecoder[-1])
    layersTransDecoder.append(unetOutput)

    modelUnetEx3 = tf.keras.Model(inputs=[LInputTarget], outputs=[unetOutput], name="Unet")

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learningRate, momentum=momentum, nesterov=False, name="SGD") 
# weight decay 0.0005 by L2

modelUnetEx3.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, 
                                                          label_smoothing=0, 
                                                          reduction="auto", 
                                                          name="binary_crossentropy"),
                  metrics=[tf.keras.metrics.BinaryCrossentropy(name="binary_crossentropy_metric")]
)

In [ ]:
# Create a TensorBoard callback
logs = "/qarr/studia/magister/models/logs/" + experimentName + datetime.now().strftime("%Y%m%d-%H%M%S")

try:
    tboard_callback = GradientsLoggerTBCallback(list(unetValidDataset.take(1))[0],
                                                                   log_dir = logs,
                                                                   histogram_freq = 1)
finally:
    pass
#except AlreadyExistsError:
#    print("Already exists, skipping")

In [ ]:
modelUnetEx3.fit(unetTrainDataset, 
              epochs=10, 
              steps_per_epoch=len(train_data_permutations)//batch_size, 
              validation_data=unetValidDataset,
              validation_steps=valid_cases//batch_size,
              callbacks=[tboard_callback]#, callbackCheckpoint]
             ) 